# Hero Overall Win Rates

Simple calculation of the overall win rate for each hero over a series of database files. Edit DB_FILES list to contain list of files.

In [1]:
import sqlite3
import os
import meta
import json
import pandas as pd
import datetime as dt
import re
import plotly.express as px
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
import numpy as np

## Merge databases and remove duplicates

In [10]:
DB_FILES=[]
for filename in os.listdir("."):    
    if re.match("^.*db$",filename) is not None:    
        DB_FILES.append(filename)
                    
rows=[]
for this_file in DB_FILES:
    conn=sqlite3.connect(this_file)
    c=conn.cursor()
    c.execute("SELECT match_id, start_time, radiant_heroes, dire_heroes, radiant_win, api_skill, items, gold_spent FROM dota_stats")
    rows.extend(c.fetchall())
    print("{0} {1}".format(this_file,len(rows)))

matches_1_all_2020082718.db 15150
matches_1_all_2020082504.db 29123
matches_1_all_2020082512.db 43909
matches_1_all_2020082520.db 60071
matches_1_all_2020082522.db 75010
matches_1_all_2020082802.db 90708
matches_1_all_2020082722.db 105975
matches_1_all_2020082814.db 108518
matches_1_all_2020082708.db 112556
matches_1_all_2020082706.db 126394
matches_1_all_2020082502.db 140019
matches_1_all_2020082508.db 154171
matches_1_all_2020082514.db 170010
matches_1_all_2020082710.db 184774
matches_1_all_2020082700.db 199414
matches_1_all_2020082506.db 213265
matches_1_all_2020082712.db 229388
matches_1_all_2020082714.db 245618
matches_1_all_2020082810.db 249470
matches_1_all_2020082806.db 264521
matches_1_all_2020082518.db 281020
matches_1_all_2020082808.db 298033
matches_1_all_2020082516.db 315823
matches_1_all_2020082804.db 330626
matches_1_all_2020082702.db 344827
matches_1_all_2020082812.db 362706
matches_1_all_2020082720.db 379465
matches_1_all_2020082500.db 393581
matches_1_all_2020082716.d

In [11]:
rows=list(set(rows))
print("Removing duplicates: {0}".format(len(rows)))

Removing duplicates: 438193


In [12]:
import mariadb

conn_maria = mariadb.connect(
    user=os.environ['DOTA_USERNAME'],
    password=os.environ['DOTA_PASSWORD'],
    host="localhost",
    database="dota")

c=conn_maria.cursor()
c.execute("select count(*) from dota_matches")
print("Rows before merge: {0} ".format(c.fetchall()[0][0]))

# Batch samples in
BATCH_SIZE=2000
for i in range(len(rows)//BATCH_SIZE+1):    
    begin=i*BATCH_SIZE
    end=min(len(rows),(i+1)*BATCH_SIZE)
    print("{0} to {1}".format(begin,end))
    #c.executemany('INSERT IGNORE INTO dota_matches (match_id, start_time, radiant_heroes, dire_heroes, radiant_win, api_skill, items, gold_spent) values(%d, %d, %s, %s, %d, %d, %s, %s)', 
    #              list(rows)[begin:end])
    c.executemany('INSERT IGNORE INTO dota_matches (match_id, start_time, radiant_heroes, dire_heroes, radiant_win, api_skill, items, gold_spent) values(?, ?, ?, ?, ?, ?, ?, ?)', 
                  list(rows)[begin:end])
    
conn_maria.commit()
c.execute("select count(*) from dota_matches")
print("Rows after merge: {0} ".format(c.fetchall()[0][0]))
conn_maria.close()

Rows before merge: 176164 
0 to 2000
2000 to 4000
4000 to 6000
6000 to 8000
8000 to 10000
10000 to 12000
12000 to 14000
14000 to 16000
16000 to 18000
18000 to 20000
20000 to 22000
22000 to 24000
24000 to 26000
26000 to 28000
28000 to 30000
30000 to 32000
32000 to 34000
34000 to 36000
36000 to 38000
38000 to 40000
40000 to 42000
42000 to 44000
44000 to 46000
46000 to 48000
48000 to 50000
50000 to 52000
52000 to 54000
54000 to 56000
56000 to 58000
58000 to 60000
60000 to 62000
62000 to 64000
64000 to 66000
66000 to 68000
68000 to 70000
70000 to 72000
72000 to 74000
74000 to 76000
76000 to 78000
78000 to 80000
80000 to 82000
82000 to 84000
84000 to 86000
86000 to 88000
88000 to 90000
90000 to 92000
92000 to 94000
94000 to 96000
96000 to 98000
98000 to 100000
100000 to 102000
102000 to 104000
104000 to 106000
106000 to 108000
108000 to 110000
110000 to 112000
112000 to 114000
114000 to 116000
116000 to 118000
118000 to 120000
120000 to 122000
122000 to 124000
124000 to 126000
126000 to 128